Summary (brief)

Prepares a 3‑qubit uniform superposition.
Implements a Grover-like step: an oracle that flips phase of state |01> (via H–CNOT–H) and a diffusion (inversion-about-mean) built with H/X and a 3‑qubit phase flip implemented as H‑Toffoli‑H (i.e., CCZ).
Applies an extra single‑qubit RZ phase on qubit 0.
Runs the circuit on a MindQuantum Simulator, obtains the statevector, samples measurement outcomes from the state probabilities, and prints counts.
The last small snippet finds the minimum of a few random numbers.

In [2]:
from mindquantum.core.gates import X, H, Z, Measure, RZ
from mindquantum.core.circuit import Circuit
from mindquantum.simulator import Simulator
import numpy as np
from collections import Counter

# Number of qubits
n_qubits = 3

# 1) Prepare uniform superposition |+++...+>
circ = Circuit()
for i in range(n_qubits):
    circ += H.on(i)   # apply H to each qubit

# 2) Oracle: flip phase of target marked state.
# Here written as CZ(control=0, target=1) implemented via H-target, CNOT(control=0,target=1), H-target.
oracle_circ = Circuit()
oracle_circ += H.on(1)
oracle_circ += X.on(1, [0])   # CNOT: X.on(target, controls)
oracle_circ += H.on(1)

# 3) Build Grover circuit = oracle + diffusion
grover_circ = Circuit() + oracle_circ

# 4) Diffusion (inversion about the mean)
#    For each qubit: H, X (map |s> to |11..1>), then apply multi-qubit Z (phase flip on |11..1>),
#    and undo with X, H. For 3 qubits we implement the multi-qubit Z as H on target + Toffoli + H.
diffusion = Circuit()
for i in range(n_qubits):
    diffusion += H.on(i)
    diffusion += X.on(i)

# implement CCZ via H on the target qubit (qubit 2), then Toffoli (X with controls [0,1] and target 2), then H
diffusion += H.on(2)
diffusion += X.on(2, [0, 1])   # Toffoli: target=2, controls=[0,1]
diffusion += H.on(2)

# undo the X/H from before to complete the inversion-about-mean
for i in range(n_qubits):
    diffusion += X.on(i)
    diffusion += H.on(i)

grover_circ += diffusion

# 5) Optional single-qubit phase rotation on qubit 0
phi = np.pi / 4
grover_circ += RZ(phi).on(0)

# 6) Execute on Simulator
sim = Simulator('mqvector_gpu', n_qubits)   # backend name may vary; ensure this backend exists
sim.apply_circuit(circ + grover_circ)       # apply the full circuit to the simulator state

# 7) Get the statevector from the simulator (try several common accessor names)
try:
    state = sim.get_qs()
except AttributeError:
    try:
        state = sim.get_statevector()
    except AttributeError:
        try:
            state = sim.get_state()
        except AttributeError as e:
            raise RuntimeError(
                "Cannot obtain statevector from Simulator. Paste full traceback if this occurs."
            ) from e

# 8) Sample measurement outcomes from statevector probabilities
shots = 100
probs = np.abs(state) ** 2                     # probability for each computational basis index
indices = np.random.choice(len(probs), size=shots, p=probs)

# Convert sample indices to bitstrings.
# NOTE: bit order depends on how the simulator indexes basis states (check MQ docs if results look reversed).
bitstrings = [format(i, f'0{n_qubits}b') for i in indices]
result = dict(Counter(bitstrings))
print(result)

# 9) Small independent snippet: repeatedly draw random numbers and keep the smallest seen.
d0 = np.random.rand()
c = 5
for _ in range(c):
    d1 = np.random.rand()
    if d1 < d0:
        d0 = d1
print('final d0:', d0)

{'111': 51, '011': 49}
final d0: 0.04215331937538802


Notes / pitfalls

Gate syntax: MindQuantum uses X.on(target, controls) (not X.on([control,target]) ). Your code uses the correct pattern.
Backend names: 'mqvector_gpu' may not exist or be unsupported on your machine; use a backend available in your env (e.g., 'projectq' or MQ default). If apply_circuit or state accessors fail, paste the exact traceback.
Bitstring ordering: when interpreting printed keys, simulator endianness matters (MSB vs LSB). If counts look reversed, swap bit order or check MQ docs.
If you want, I can:

Add explicit measurement-by-circuit (if your MQ version supports it), or
Replace the Toffoli decomposition with a 2-qubit decomposition that avoids multi-controlled X if your MQ version lacks that support.